In [2]:
!pip install ultralytics opencv-python


In [17]:
!wget https://raw.githubusercontent.com/abewley/sort/master/sort.py


--2025-07-08 09:32:43--  https://raw.githubusercontent.com/abewley/sort/master/sort.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11739 (11K) [text/plain]
Saving to: ‘sort.py.2’

sort.py.2           100%[===================>]  11.46K  --.-KB/s    in 0s      

2025-07-08 09:32:43 (70.5 MB/s) - ‘sort.py.2’ saved [11739/11739]



In [27]:
# Remove matplotlib imports
!sed -i '/import matplotlib/d' sort.py
!sed -i '/from matplotlib/d' sort.py
# Remove matplotlib.use line
!sed -i '/matplotlib.use/d' sort.py


In [29]:
!pip install filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=db6a280d663fdb5bcd0c4fdcc21a345c48e2a67191832d2311dc2dc18191d33c
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [30]:
from sort import Sort


YOLO: Loads your object detection model.

cv2: For working with video and drawing boxes.

np: For numeric operations.

Sort: A simple tracking library that automatically assigns IDs.

In [31]:
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO('/content/best.pt')


This command downloads sort.py file, which contains the SORT tracker code.

SORT will track players across frames and assign IDs automatically.

In [32]:
!wget https://raw.githubusercontent.com/abewley/sort/master/sort.py


--2025-07-08 09:45:16--  https://raw.githubusercontent.com/abewley/sort/master/sort.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11739 (11K) [text/plain]
Saving to: ‘sort.py.3’

sort.py.3           100%[===================>]  11.46K  --.-KB/s    in 0s      

2025-07-08 09:45:16 (75.9 MB/s) - ‘sort.py.3’ saved [11739/11739]



In [35]:
# Initialize tracker
tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3)

# Open video
video_path = '/content/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run detection
    results = model(frame)

    # Prepare detections for tracker (x1, y1, x2, y2, score)
    detections = []
    for box, conf, cls in zip(results[0].boxes.xyxy.cpu().numpy(),
                              results[0].boxes.conf.cpu().numpy(),
                              results[0].boxes.cls.cpu().numpy()):
        # Only process if it's a player class (check your class ID if needed)
        detections.append([box[0], box[1], box[2], box[3], conf])

    dets = np.array(detections)

    # Handle case where no detections are made
    if dets.size == 0:
        tracks = tracker.update(np.empty((0, 5))) # Pass empty array with expected shape
    else:
        tracks = tracker.update(dets)


    # Draw results
    for track in tracks:
        x1, y1, x2, y2, track_id = track.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Save or display
    cv2.imwrite(f"/content/frame_{frame_count:04d}.jpg", frame)

cap.release()


0: 384x640 1 ball, 16 players, 2 referees, 2342.0ms
Speed: 5.1ms preprocess, 2342.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2956.2ms
Speed: 2.9ms preprocess, 2956.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2768.4ms
Speed: 3.0ms preprocess, 2768.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2380.8ms
Speed: 2.7ms preprocess, 2380.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2377.3ms
Speed: 2.6ms preprocess, 2377.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2339.2ms
Speed: 3.0ms preprocess, 2339.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 3158.7ms
Speed: 4.2ms preprocess, 3158.7ms inference, 1.7ms 

In [36]:
import cv2
import glob

# Get list of saved frames
frame_files = sorted(glob.glob('/content/frame_*.jpg'))

# Read first frame to get size
frame = cv2.imread(frame_files[0])
height, width, _ = frame.shape

# Create video writer
out = cv2.VideoWriter('/content/final_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

# Write all frames to video
for file in frame_files:
    img = cv2.imread(file)
    out.write(img)

out.release()
print("✅ Video saved as /content/final_output.mp4")


✅ Video saved as /content/final_output.mp4


In [37]:
from google.colab import files
files.download('/content/final_output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>